In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import HashingVectorizer


from tqdm.auto import tqdm, trange

# Считываю дф улиц. добавляю дф с двумя столбцами

In [3]:
test_df_two_col = pd.read_excel('clear drop nan sh+city and sh + streat + type.xlsx')

In [4]:
test_df_two_col.head(10)

,concat_name,type,S6_ID_str
0,автодорога екад пр-кт академика сахарова,автодорога екад пр-кт академика сахарова,838
1,автодорога екад ул гоголя,автодорога екад ул гоголя,40452
2,автодорога екад ул курганская,автодорога екад ул курганская,25047
3,автодорога екад ул уральских рабочих,автодорога екад ул уральских рабочих,40453
4,автодорога екад ул чайковского,автодорога екад ул чайковского,40454
5,автодорога екад ул черепанова,автодорога екад ул черепанова,25514
6,автодорога екад ул эскадронная,автодорога екад ул эскадронная,40455
7,автодорога екатеринбург - аэропорт кольцово пе...,автодорога екатеринбург - аэропорт кольцово пе...,40459
8,автодорога екатеринбург - аэропорт кольцово ул...,автодорога екатеринбург - аэропорт кольцово ул...,40456
9,автодорога екатеринбург - аэропорт кольцово ул...,автодорога екатеринбург - аэропорт кольцово ул...,40457


In [11]:
X = test_df_two_col['concat_name']
y = test_df_two_col['type'] + '++' +test_df_two_col['S6_ID_str']
# Добавляю сразу S6 чтобы после не перепутать какое название в какому ID относится

In [12]:
y

0        автодорога екад пр-кт академика сахарова++838
1                     автодорога екад ул гоголя++40452
2                 автодорога екад ул курганская++25047
3          автодорога екад ул уральских рабочих++40453
4                автодорога екад ул чайковского++40454
                             ...                      
31264                  х николаев ул набережная++18396
31265            х новосергиевка ул центральная++14410
31266                     х петровка ул полевая++16842
31267               х расстригин ул центральная++41268
31268                   х тупилкин ул береговая++13684
Length: 31269, dtype: object

# Добавляю тестовую выборку

## Данные на которых я обучал ранее

In [3]:
df_test_clear = pd.read_excel('test_clean_num_empty_07.06.xlsx')

In [13]:
df_test_clear.shape

(60624, 4)

In [6]:
df_test_clear['test_concat'] = df_test_clear['NASP_cler'] + ' ' + df_test_clear['ULIC_cler']

In [7]:
df_test_clear.dropna().to_excel('test_clean_num_empty_dropna_11.06 .xlsx')

In [14]:
df_test_clear

,NASP_cler,ULIC_cler,ID,test_concat
0,штат техас,драйв кингфишер,3093,штат техас драйв кингфишер
1,обл московская,коломийца,3094,обл московская коломийца
2,германия,нолдеринг,3117,германия нолдеринг
3,обл курганская,ул савельева,3844,обл курганская ул савельева
4,хмао-югра,район упр-2,7370,хмао-югра район упр-2
...,...,...,...,...
60619,область свердловская,ул заславского,23223,область свердловская ул заславского
60620,г серов,ул заславского,35101,г серов ул заславского
60621,область свердловская,ул заславского,35102,область свердловская ул заславского
60622,серов,заславского,40513,серов заславского


## Данные с заменой "свердловской области" на "Екатеринбург" 

In [31]:
df_test_clear = pd.read_excel('test_clean_num_empty_09.06 с заменой сверд. обл. на екатеринбург .xlsx')

In [32]:
df_test_clear['test_concat'] = df_test_clear['NASP_cler'] + ' ' + df_test_clear['ULIC_cler']

In [33]:
test_srt_concat = df_test_clear['test_concat']

In [36]:
df_test_clear.to_excel('test_clean_num_empty_09.06 с заменой сверд. обл. на екатеринбург .xlsx')

In [25]:
test_srt_concat = test_srt_concat.dropna()

In [26]:
test_srt_concat.shape

(60596,)

In [27]:
test_srt_concat_1 = test_srt_concat[:30000]
test_srt_concat_2 = test_srt_concat[30000:]

# Naive Bayes Classifier

Naive Bayes Classifier

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

## Рабочий вариант 95% точность. Попытка 2

In [14]:
nb = Pipeline([('vect', CountVectorizer(analyzer='char', ngram_range=(3, 4), binary = True)),
#                ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [15]:
%%time
nb.fit(X, y)

Wall time: 1min 37s


Pipeline(steps=[('vect',
                 CountVectorizer(analyzer='char', binary=True,
                                 ngram_range=(3, 4))),
                ('clf', MultinomialNB())])

In [18]:
y_pred = nb.predict(test_srt_concat_1)

probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_1)).max(axis=1)

pred_df = pd.DataFrame(test_srt_concat_1).reset_index(drop=True)

pred_df['pred'] = y_pred

pred_df = pred_df.reset_index(drop=True)

pred_df['probability'] = probability_pred_1

# Вторая часть предсказанных данных

y_pred = nb.predict(test_srt_concat_2)

probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_2)).max(axis=1)

pred_df_2 = pd.DataFrame(test_srt_concat_2).reset_index(drop=True)

pred_df_2['pred'] = y_pred

pred_df_2 = pred_df_2.reset_index(drop=True)

pred_df_2['probability'] = probability_pred_1

In [19]:
pd.concat([pred_df, pred_df_2]).to_excel('MultinomialNB probability all + замена обл.  analyzer=char, ngram_range=(3, 4).xlsx')